In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30, 
    brightness_range=[0.8, 1.2],
    validation_split=0.2  
)

In [3]:
data_directory = r"C:\Users\admin\Desktop\Projects\CS-Project\Tea-Leaf-Comprehensive-Anaysis\Data\nd-image" 

In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical',  
    subset='training'  
)

In [ ]:
# Validation Data Generator
validation_generator = train_datagen.flow_from_directory(
    data_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  
)

In [6]:
# Load the pre-trained VGG16 model without the top layers
base_model = tf.keras.applications.VGG16(
    weights='imagenet',    
    include_top=False,     
    input_shape=(224, 224, 3) 
) 

In [7]:
base_model.trainable = False

In [8]:
model = tf.keras.Sequential()

In [9]:
model.add(base_model)

In [10]:
from tensorflow.keras import regularizers


model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation = 'relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='softmax'))

In [11]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [13]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data = validation_generator,
    callbacks= [early_stop]
)

In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [18]:
model.save('nitrogen_deficiency_classifier_vgg16.keras')

In [ ]:
def predict_deficiency(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.  

    prediction = model.predict(img_array)
    categories = ['Healthy', 'Early', 'Advanced']

    predicted_class = np.argmax(prediction[0])
    
    print(f"Predicted Category: {categories[predicted_class]}")


predict_deficiency('Test2.jpeg')  